In [ ]:
from google.colab import files
uploaded = files.upload()


Saving healthcare-dataset-stroke-data.csv to healthcare-dataset-stroke-data (1).csv


In [ ]:
import pandas as pd

df = pd.read_csv("healthcare-dataset-stroke-data.csv")
df.head()


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [ ]:
# Select categorical columns to encode
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Perform one-hot encoding
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print("DataFrame after one-hot encoding:")
display(df.head())

DataFrame after one-hot encoding:


,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,9046,67.0,0,1,228.69,36.6,1,True,False,True,False,True,False,False,True,True,False,False
1,51676,61.0,0,0,202.21,NaN,1,False,False,True,False,False,True,False,False,False,True,False
2,31112,80.0,0,1,105.92,32.5,1,True,False,True,False,True,False,False,False,False,True,False
3,60182,49.0,0,0,171.23,34.4,1,False,False,True,False,True,False,False,True,False,False,True
4,1665,79.0,1,0,174.12,24.0,1,False,False,True,False,False,True,False,False,False,True,False


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Separate features (X) and target variable (y)
X = df.drop('stroke', axis=1)
y = df['stroke']

# Instantiate and train the RandomForestClassifier model
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

# Get feature importances
feature_importances = pd.Series(model.feature_importances_, index=X.columns)

# Sort feature importances in descending order
sorted_feature_importances = feature_importances.sort_values(ascending=False)

# Display feature importances
print("Feature Importances:")
display(sorted_feature_importances)

Feature Importances:


,0
avg_glucose_level,2.086011e-01
id,2.072990e-01
age,1.899025e-01
bmi,1.841719e-01
Residence_type_Urban,2.806962e-02
gender_Male,2.590761e-02
hypertension,2.480345e-02
heart_disease,2.337326e-02
smoking_status_formerly smoked,2.062636e-02
smoking_status_never smoked,1.958510e-02


In [ ]:
# Set a threshold for feature importance
importance_threshold = 0.05 # You can adjust this threshold

# Select features based on the threshold
selected_features = sorted_feature_importances[sorted_feature_importances > importance_threshold].index.tolist()

print(f"Selected features (importance > {importance_threshold}):")
print(selected_features)

# Create a new DataFrame with only the selected features and the target variable
# Use 'stroke' as the target column as it was not one-hot encoded
if 'stroke' in df.columns:
    df_selected = df[selected_features + ['stroke']]
else:
    print("Error: 'stroke' column not found in the DataFrame.")
    df_selected = None


if df_selected is not None:
    print("\nShape of DataFrame with selected features:", df_selected.shape)
    display(df_selected.head())

Selected features (importance > 0.05):
['avg_glucose_level', 'id', 'age', 'bmi']

Shape of DataFrame with selected features: (5110, 5)


,avg_glucose_level,id,age,bmi,stroke
0,228.69,9046,67.0,36.6,1
1,202.21,51676,61.0,NaN,1
2,105.92,31112,80.0,32.5,1
3,171.23,60182,49.0,34.4,1
4,174.12,1665,79.0,24.0,1
